In [1]:
import mysql.connector

In [4]:
mydb=mysql.connector.connect(host="localhost",port=3306,user="root",password="",database="adbms_final")
print(mydb)

In [5]:
mycursor = mydb.cursor(buffered=True)

In [6]:
mycursor.execute("CREATE TABLE instructor (ID int,name VARCHAR(255), dept_name VARCHAR(255), salary int)")
mydb.commit()

In [8]:
mycursor.execute("SHOW TABLES")
for x in mycursor:
    print(x)

('instructor',)


In [9]:
sql = "INSERT INTO instructor (id, name, dept_name, salary) VALUES (%s, %s,%s, %s)"
val = [
    (10101, "Srinivasan", "Comp. Sci.", 65000),
    (12121, "Wu", "Finance", 90000),
    (15151, "Mozart", "Music", 40000),
    (22222, "Einstein", "Physics", 95000),
    (32343, "El Said", "History", 60000),
    (33456, "Gold", "Physics", 87000),
    (45565, "Katz", "Comp. Sci.", 75000),
    (58583, "Califieri", "History", 62000),
    (76543, "Singh", "Finance", 80000),
    (76766, "Crick", "Biology", 72000),
    (83821, "Brandt", "Comp. Sci.", 92000),
    (98345, "Kim", "Elec. Eng.", 80000)
]
mycursor.executemany(sql, val)
mydb.commit()

In [10]:
mycursor.execute("ALTER TABLE instructor ADD PRIMARY KEY (ID)")
mydb.commit()

Create a view of instructors without their salary called faculty

In [12]:
mycursor.execute("CREATE VIEW Faculty AS SELECT id, name, dept_name, salary FROM instructor")
mydb.commit()

Create a view of department salary totals

In [13]:
mycursor.execute("CREATE VIEW DeptSalaryTotals AS SELECT dept_name,sum(salary) FROM instructor group by dept_name")
mydb.commit()

Create a role of student

In [32]:
mycursor.execute("CREATE ROLE student")
mydb.commit()

Give select privileges on the view faculty to the role student

In [34]:
mycursor.execute("GRANT SELECT on adbms_final.faculty to student")
mydb.commit()

Create a new user and assign her the role of student.

In [35]:
mycursor.execute("create user 'new_user'@'localhost' identified by ''")
mycursor.execute("grant student to 'new_user'@'localhost'")
mydb.commit()

Revoke privileges of the new user

In [36]:
mycursor.execute("REVOKE ALL PRIVILEGES ON *.* FROM 'new_user'@'localhost'")
mydb.commit()

Remove the role of student

In [37]:
mycursor.execute("drop role student")
mydb.commit()

Give select privileges on the view faculty to the new user.

In [39]:
mycursor.execute("grant select on adbms_final.faculty to 'new_user'@'localhost'")
mydb.commit()

Create table teaches2 with same columns as teaches but with additional constraintthat that semester is one of fall, winter, spring or summer.

In [46]:
mycursor.execute("""
            CREATE TABLE teaches2 (
                ID INT,
                Course_id VARCHAR(255),
                sec_id INT,
                semester ENUM('fall', 'winter', 'spring', 'summer'),
                PRIMARY KEY (ID)
            )
        """)
mydb.commit()

Create index ID column of teaches. Compare the difference in time to obtain query
results with or without index.

In [48]:
mycursor.execute("create index idx_teaches_ID on teaches(ID)")
mydb.commit()

In [50]:
import time
startime=time.time()
sql="select * from teaches where id=32343"
mycursor.execute(sql)
myresult = mycursor.fetchall()
for x in myresult:
    print(x)
endtime=time.time()
duration=endtime-startime
print("Time without using index: ",duration)

(32343, 'HIS-351', 1, 'Spring', 2018)
Time without using index:  0.002965688705444336


In [52]:
import time
startime=time.time()
sql="SELECT * FROM teaches USE INDEX (idx_teaches_ID) WHERE ID = 32343"
mycursor.execute(sql)
myresult = mycursor.fetchall()
for x in myresult:
    print(x)
endtime=time.time()
duration=endtime-startime
print("Time with using index: ",duration)

(32343, 'HIS-351', 1, 'Spring', 2018)
Time with using index:  0.00099945068359375
